In [1]:
from docplex.mp.model import Model

# Sets
S = 2  # Set of suppliers
W = 5  # Set of workshops
R = 2  # Set of raw materials
C = 1  # Construction site

# Distances
d_sw = [[5, 2, 4.1, 6, 4, 9], [5.8, 3.9, 2.3, 8, 4.2]]  # Distance from supplier s to workshop w
d_ww = [[0, 0.9, 3.8, 1, 8], [0.9, 0, 1, 3.5, 2], [3.8, 1, 0, 7, 1.2], [1, 3.5, 7, 0, 15], [8, 2, 1.2, 15, 0]]  # Distance from workshop w to workshop wprime
d_wc = [4, 3, 4.5, 6, 5.8]  # Distance from workshop w to construction site c

# Raw materials
r_ms = [[1, 0], [0, 1]]  # Binary variable equal to 1 if supplier s has raw material m
w_ms = [[1, 1], [1, 1], [1, 1], [1, 0], [0, 1]]  # Binary variable equal to 1 if workshop j requires raw material m

# Create a new model
mdl = Model("Shuttle")

# Set the time limit to 86400 seconds
mdl.set_time_limit(86400)

# Set the relative tolerance to 1
mdl.parameters.mip.tolerances.mipgap = 1


# Decision variables
x_s = [[mdl.binary_var(name='x_s_{}_{}'.format(s+1, k+1)) for k in range(2)] for s in range(S)]  # Binary variable equal to 1 if the truck visits supplier s in shuttle k
x_w = [[mdl.binary_var(name='x_w_{}_{}'.format(w+1, k+1)) for k in range(2)] for w in range(W)]  # Binary variable equal to 1 if the truck visits workshop w in shuttle k
x_ww = [[[mdl.binary_var(name='x_ww_{}_{}_{}'.format(w+1, w_prime+1, k+1)) for k in range(2)] for w_prime in range(W)] for w in range(W)]  # Binary variable equal to 1 if the truck travels from workshop w to workshop wprime in shuttle k
x_c = [mdl.binary_var(name='x_c_{}_{}'.format(c+1, k+1)) for c in range(C) for k in range(2)]  # Binary variable equal to 1 if the truck visits construction site c in shuttle k
y_sw = [[[mdl.binary_var(name='y_sw_{}_{}_{}'.format(s+1, w+1, k+1)) for k in range(2)] for w in range(W)] for s in range(S)]  # Binary variable equal to 1 if shuttle k transports raw material from supplier s to workshop w

# Objective function
mdl.minimize(mdl.sum(mdl.sum(mdl.sum(d_sw[s][w]*x_s[s][k] for s in range(S)) for w in range(W)) for k in range(2))
             + mdl.sum(mdl.sum(mdl.sum(d_ww[w][w_prime]*x_ww[w][w_prime][k] for w in range(W)) for w_prime in range(W)) for k in range(2))
             + mdl.sum(mdl.sum(d_wc[w]*x_w[w][k] for w in range(W)) for k in range(2)))

# Constraints
for s in range(S):
    mdl.add_constraint(mdl.sum(x_s[s][k] for k in range(1)) == 1)  # Each supplier is visited once and only once in the first shuttle.
    mdl.add_constraint(x_s[s][1] == 0)  # Only the first shuttle visits suppliers.

for w in range(W):
    mdl.add_constraint(mdl.sum(x_w[w][k] for k in range(2)) >= 1)  # Each workshop is visited once and only once in each shuttle.
    for k in range(2):
        mdl.add_constraint(x_ww[w][w][k] == 0)  # Diagonal constraint.

for m in range(R):
    for w in range(W):
        mdl.add_constraint(mdl.sum(r_ms[m][s]*x_s[s][0] for s in range(S)) >= w_ms[w][m]*x_w[w][0])  # Before visiting a workshop, the truck must have the required raw materials.

for c in range(C):
    mdl.add_constraint(x_c[c] == 1)  # The construction site is visited by the second shuttle.
    mdl.add_constraint(mdl.sum(x_w[w][1] for w in range(W)) == 1)  # The construction site is the final destination of the second shuttle.

# Solve the model
solution = mdl.solve()


In [2]:
# Print the solution
print("Total distance of both shuttles: ", solution.objective_value)

print("Total distance of first shuttle: ", sum(sum(d_sw[s][w]*solution.get_value(x_s[s][0]) for s in range(S)) for w in range(W))
      + sum(sum(d_ww[w][w_prime]*solution.get_value(x_ww[w][w_prime][0]) for w in range(W)) for w_prime in range(W))
      + sum(d_wc[w]*solution.get_value(x_w[w][0]) for w in range(W)))

print("The visit order of first shuttle: ", [s+1 for s in range(S) if solution.get_value(x_s[s][0]) == 1], "->",
      [w+1 for w in range(W) if solution.get_value(x_w[w][0]) == 1])
print("Total distance of second shuttle: ", sum(sum(d_sw[s][w]*solution.get_value(x_s[s][1]) for s in range(S)) for w in range(W))
      + sum(sum(d_ww[w][w_prime]*solution.get_value(x_ww[w][w_prime][1]) for w in range(W)) for w_prime in range(W))
      + sum(d_wc[w]*solution.get_value(x_w[w][1]) for w in range(W)))
print("The visit order of second shuttle: ", [w+1 for w in range(W) if solution.get_value(x_w[w][1]) == 1], "->",
      [c+1 for c in range(C) if solution.get_value(x_c[c]) == 1])


Total distance of both shuttles:  72.6
Total distance of first shuttle:  68.6
The visit order of first shuttle:  [1, 2] -> [1, 2, 3, 4, 5]
Total distance of second shuttle:  4.0
The visit order of second shuttle:  [1] -> [1]
